# Aplicando IBM IOCR em Arquiteturas RAG: Como Potencializar a Extração de Texto

### Instalação dos pacotes necessários para a execução do código

In [1]:
pip install ibm-watsonx-ai==1.1.5 -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ibm-cos-sdk==2.12.0 -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ibm_watson_machine_learning -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-dotenv -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pytesseract -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Configuração das variáveis de ambiente

Antes de prosseguir, configure as variáveis de ambiente criando um arquivo .env no mesmo diretório de execução do notebook

#### PATH_IMAGE: caminho do diretório da sua imagem


#### WATSONX_API_KEY: API KEY da IBM Cloud, pode ser obtida conforme o passo a passo abaixo:

<img src="images/apikey_1.png" alt="wxd-1"  />

<img src="images/apikey_2.png" alt="api2"  />

<img src="images/apikey_3.png" alt="api3"  />

#### WATSONX_PROJECT_ID: ID do seu projeto no watsonx ai, o qual pode ser obtido nas configurações do projeto


<img src="images/wx-id.png" alt="wx-id"  />

#### Credenciais do COS

Uma vez na intância do COS, entre em Service credentials e gere uma nova com o Role "Manager" e com a opção HMAC ativa.

<img src="images/service_cred_1.png" alt="serv-1"  />

<img src="images/service_cred_3.png" alt="serv-2"  />

Em seguida, colote: COS_API_KEY, COS_ACCESS_KEY_ID e COS_SECRET_ACCESS_KEY

<img src="images/service_cred_4.png" alt="serv-3"  />

Crie um bucket que será o armazenamento da imagem e texto extraído. Após criado, preencha seu .env com o nome do bucket.

<img src="images/bucket_1.png" alt="bucket-1"  />

<img src="images/bucket_2.png" alt="bucket-2"  />

<img src="images/bucket_3.png" alt="bucket-3"  />

# Ponto de partida para comparação de modelos


Vamos utilizar a divulgação de resultados da Magalu como base para alimentar o modelo de IA generativa, permitindo que ele responda a perguntas específicas sobre a empresa. 

A escolha desse documento é intencional, pois ele reúne uma grande quantidade de informações estruturadas, como tabelas, indicadores financeiros e dados operacionais relevantes.

<img src="images/RI-Magalu.png" alt="magalu"  />

### Pytesseract - OCR OpenSource

Importa bibliotecas necessárias para OCR (Reconhecimento Óptico de Caracteres) e manipulação de imagens, além de carregar variáveis de ambiente.

Bibliotecas: 
- pytesseract: OCR open source
- PIL (Python Imaging Library): Manipulação de imagens
- dotenv: Carregar variáveis de ambiente
- os: Interação com o sistema operacional

In [6]:
import pytesseract
from PIL import Image
from dotenv import load_dotenv
import os

load_dotenv()

True

Recuperando o caminho do diretório de imagens a partir de uma variável de ambiente

In [7]:
path_image = os.getenv("PATH_IMAGE")

Extração de texto de imagem utilizando OCR (Optical Character Recognition), verifica se o arquivo de imagem existe, abre a imagem, aplica OCR e salva o resultado em uma pasta temporária.

Podemos observar o texto extraido pela aplicação de um OCR open source

In [8]:
if not os.path.isfile(path_image):
    raise FileNotFoundError(f"Arquivo não encontrado: {path_image}")

image = Image.open(path_image)


temp_folder = "temp_images"
os.makedirs(temp_folder, exist_ok=True)


saved_image_path = os.path.join(temp_folder, "processed_image.png")

image.save(saved_image_path, "PNG")

ocr_text = pytesseract.image_to_string(image)

print("\nTexto extraído da imagem:\n", ocr_text)


Texto extraído da imagem:
 Divulgagio de Resultados

4724

magaLu

RS milhbes(exceto quando indicado) 723 Var(X)—2M24_2M23_Vvarix)
\VendasTotas (incuindo marketplace) 1saig9 179474 26% 65.3309 63.0553 36K.
Receita ruta 13aog. —13082,5 26% «47.2770 «A550 37%.
Receita Liquida 107873 10543,7 23% 3803813676813 5H.
wero Bruto 3200 «3.1928 16% 11627,3 10.1639 14a
Margem Bruta 301% 303% 0.2pp 306% 276% 3,0
EBITDA 24 506 536% 2.895,7 8705 232.6%
Margem EBITDA 78% 52%  26pp 75% 24% 52pp
Luero Liquide 2348 222 38.9% 437 (9731) .
Margem Liquids ™% 9% 07 pp 42% 27% __39pp
Lucro Bruto -Ajustad 3200 «3.1928 16% = 11627,3 10750182
Margem ruta - Austad 301% 303% 0.2pp 306% 282% 14pp
EBITDA -Ajustado 46,2 7555 149% «29622 231B 380K
Margem EBITDA Ajustado 78% 72% Opp 73% 58% 20pp
Luero Liquide - Austad 1392 11s 37% 2767 —_(550,1) .
Margem Liquida- Ajustado 13% 10% __0.3pp 07% 15% PD.
Crescimento nas Vendas Mesmas Loja Fisicat 31% 3% 2 17% 47% zs
Crescimento nas Vendas Tota LojasFsicas 64% 3.5% 2 102% 4

## Perguntas e Respostas (Q&A) do artigo a partir do texto extraido pelo Pytesseract


### Configuração da API do watsonx.ai

Configuração do modelo de IA generativa para processamento de linguagem natural.

Importa a classe Model do pacote foundation_models do IBM Watson Machine Learning.

Define as credenciais e parâmetros para acesso ao modelo Meta Llama 3.1 70B Instruct.

In [9]:
from ibm_watson_machine_learning.foundation_models import Model

WATSONX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

model_id = 'meta-llama/llama-3-1-70b-instruct'

parameters = {
        "decoding_method": "greedy",
        "max_new_tokens": 1200,
        "stop_sequences": None,
        "repetition_penalty": 1
}

model_gen_ai = Model(
        model_id = model_id,
        params = parameters,
        credentials = {
            "url" : "https://us-south.ml.cloud.ibm.com",
            "apikey" : WATSONX_API_KEY
        },
        project_id = WATSONX_PROJECT_ID,
)


/Users/laurapellizaripereira/Downloads/Build-Blocks/ocr/env/lib/python3.12/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'meta-llama/llama-3-1-70b-instruct' is in deprecated state from 2025-01-22 until 2025-05-30. IDs of alternative models: llama-3-3-70b-instruct, llama-3-2-90b-vision-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


###  Geração de resposta a partir de um modelo de IA generativo para pergunta com base no texto extraído via OCR

In [10]:
input_question = "Qual foi o EBITDA em 4T23? "  

prompt_text = f'''  # Construção do prompt para o modelo de IA
        Você é um assistente responsável por responder perguntas com base em um contexto fornecido. 
        Este é seu contexto: {ocr_text}.
        Pergunta: {input_question} 
        Resposta:'''

result = model_gen_ai.generate_text(prompt=prompt_text)

In [11]:
print(result)

 24.506


##### Repare na imagem que a resposta correta seria 548,6 milhões.

# IBM OCR for documents (IOCR)

## Configuração do ambiente

Antes de usar o código de exemplo deste notebook, você deve realizar as seguintes tarefas de configuração:

- Crie uma instância do <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">serviço Watson Machine Learning (WML)</a> (um plano gratuito é oferecido e informações sobre como criar a instância podem ser encontradas <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">aqui</a>).


## Definição das credenciais do WML
Esta célula define as credenciais do WML necessárias para trabalhar com inferência do Modelo de Fundação do watsonx.
 
**Ação:** Forneça a chave de API do usuário da IBM Cloud. Para detalhes, consulte <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentação</a>.


In [10]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(url="https://us-south.ml.cloud.ibm.com",
                          api_key=WATSONX_API_KEY)


### Definição do ID do projeto
O serviço de Extração de Texto requer um ID de projeto que forneça o contexto para a chamada. 

Obteremos o ID do projeto em que este caderno é executado. 

Caso contrário, forneça o ID do projeto.

In [11]:
try:
    project_id = WATSONX_PROJECT_ID
except KeyError:
    project_id = input("Por favor, insira seu project_id: ")


### Inicialização do cliente da API


Importa a classe APIClient do módulo ibm_watsonx_ai para interagir com o watsonx.ai

Instancia a classe APIClient com as credenciais e o ID do projeto fornecidos

In [12]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials=credentials, project_id=project_id)

## Criar conexões de dados com documento de origem e referência de resultados


O documento do qual vamos extrair texto está localizado no IBM Cloud Object Storage (COS). 

No exemplo a seguir, usaremos a mesma imagem sobre resultados financeiros da Magalu que foi utilizado no primeiro exemplo extraido por uma biblioteca opensource.

Além disso, o arquivo de resultados finais, que conterá o texto extraído e metadados necessários, será colocado no COS. 
Portanto, usamos as classes `ibm_watsonx_ai.helpers.DataConnection` e `ibm_watsonx_ai.helpers.S3Location` para criar objetos Python que representarão as referências aos arquivos processados.

Observação: é necessário criar um ativo de conexão com os detalhes do seu COS 

(para uma explicação detalhada de como fazer isso, consulte [conexão do IBM Cloud Object Storage ](https://dataplatform.cloud.ibm.com/docs/content/wsj/manage-data/conn-cos.html?context=wx) ou verifique as células abaixo).



### Crie uma conexão com o COS
Você pode pular esta seção se já tiver um ativo de conexão com o **Cloud Object Storage IBM Cloud**.


In [13]:
COS_BUCKET_NAME = os.getenv("COS_BUCKET_NAME")
COS_API_KEY = os.getenv("COS_API_KEY")
COS_ACCESS_KEY_ID = os.getenv("COS_ACCESS_KEY_ID")
COS_SECRET_ACCESS_KEY = os.getenv("COS_SECRET_ACCESS_KEY")

In [14]:
datasource_name = 'cloudobjectstorage'
bucketname = COS_BUCKET_NAME

Definição das credenciais para acesso ao Cloud Object Storage (COS) da IBM

As variáveis COS_API_KEY, COS_ACCESS_KEY_ID e COS_SECRET_ACCESS_KEY devem ser previamente configuradas no arquivo .env

Essas credenciais são utilizadas para autenticar e autorizar o acesso ao COS.

In [15]:
cos_credentials = {
    "endpoint_url": "https://s3.us-south.cloud-object-storage.appdomain.cloud",
    "apikey": COS_API_KEY,
    "access_key_id": COS_ACCESS_KEY_ID,
    "secret_access_key": COS_SECRET_ACCESS_KEY
}

In [16]:
conn_meta_props= {
    client.connections.ConfigurationMetaNames.NAME: f"Connection to Database - {datasource_name} ",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: client.connections.get_datasource_type_id_by_name(datasource_name),
    client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to external Database",
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': bucketname,
        'access_key': cos_credentials['access_key_id'],
        'secret_key': cos_credentials['secret_access_key'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',
        'url': cos_credentials['endpoint_url']
    }
}


conn_details = client.connections.create(meta_props=conn_meta_props)
connection_asset_id = client.connections.get_id(conn_details)

Creating connections...
SUCCESS



### Faça o upload do arquivo e crie a referência do documento e dos resultados


Configuração de conexão com armazenamento em nuvem para dados de entrada e saída

In [ ]:
from ibm_watsonx_ai.helpers import DataConnection, S3Location


image_name = "RI-Magalu" 

source_file_name = image_name + ".png" 
results_file_name = "plain.txt"


Estabelece uma conexão remota para gravar um arquivo em um local específico da estrutura de armazenamento em nuvem S3 e publica o resultado.

In [19]:
remote_document_reference = DataConnection(connection_asset_id=connection_asset_id,
                                           location=S3Location(bucket = bucketname, path = "."))

# Define o cliente para a conexão remota
remote_document_reference.set_client(client)

# Grava o arquivo no local remoto especificado
remote_document_reference.write(path_image, remote_name=source_file_name)


### Criar uma conexão de dados que represente o documento e a referência de resultados.


Definição das referências de dados para conexão com o S3

In [ ]:
document_reference = DataConnection(
    connection_asset_id=connection_asset_id,
    location=S3Location(
        bucket=bucketname,
        path=source_file_name
    )
)

results_reference = DataConnection(
    connection_asset_id=connection_asset_id,
    location=S3Location(
        bucket=bucketname,
        path=results_file_name
    )
)


## Extrair texto


Como as conexões de dados para arquivos de origem e resultados estão prontas, podemos prosseguir para a etapa do trabalho de execução da extração de texto.

Para inicializar o gerenciador de extração de texto, usamos a classe `TextExtractions`.


In [21]:
from ibm_watsonx_ai.foundation_models.extractions import TextExtractions
from ibm_watsonx_ai.metanames import TextExtractionsMetaNames

Inicializa a extração de texto utilizando a API client e o ID do projeto

In [22]:
extraction = TextExtractions(api_client=client, project_id=project_id)

Ao executar o trabalho, é possível especificar as etapas do pipeline de extração de texto.

Para obter mais detalhes sobre as etapas disponíveis, consulte a [documentação](https://cloud.ibm.com/apidocs/watsonx-ai#text-extraction).

In [23]:
TextExtractionsMetaNames().show()

----------------  ----  --------
META_PROP NAME    TYPE  REQUIRED
OCR               dict  N
TABLE_PROCESSING  dict  N
----------------  ----  --------


Para visualizar valores de parâmetros de amostra para as etapas de extração de texto, execute `get_example_values()`.

In [24]:
TextExtractionsMetaNames().get_example_values()

{'ocr': {'process_images': True, 'language_list': ['en']},
 'tables_processing': {'enabled': True}}

No código de exemplo a seguir, utilizaremos os passos abaixo


In [25]:
steps = {
    "ocr": {
        "languages_list": [
            "pt"
        ]
    },
    "tables_processing": {
        "enabled": True
    },
    "assembly_md": {}
}


Agora podemos executar o trabalho de extração de texto.


In [38]:
details = extraction.run_job(document_reference=document_reference, 
                             results_reference=results_reference, 
                             steps=steps)
print(details)


{'metadata': {'id': '69cc5101-8ea6-49c9-8fb8-8b8d984a2c02', 'created_at': '2025-05-13T20:29:02.927Z', 'project_id': '3bb56b5d-0778-4f71-bd40-d6d7d5b9011f'}, 'entity': {'document_reference': {'type': 'connection_asset', 'connection': {'id': '8a9a4e88-24df-4ddd-8ef1-7e1dc95b19b4'}, 'location': {'bucket': 'bucket-uxe54qdrdqc2c41', 'file_name': 'RI-Magalu.png'}}, 'results_reference': {'type': 'connection_asset', 'connection': {'id': '8a9a4e88-24df-4ddd-8ef1-7e1dc95b19b4'}, 'location': {'bucket': 'bucket-uxe54qdrdqc2c41', 'file_name': 'plain.txt'}}, 'steps': {'ocr': {'languages_list': ['pt']}, 'tables_processing': {'enabled': True}}, 'parameters': {'requested_outputs': ['plain_text'], 'mode': 'standard', 'languages': ['pt'], 'create_embedded_images': 'disabled', 'output_dpi': 72, 'output_tokens_and_bbox': True}, 'results': {'status': 'submitted', 'number_pages_processed': 0}}}


Obtém o ID do trabalho de extração com base nos detalhes fornecidos

In [39]:
extraction_job_id = extraction.get_id(extraction_details=details)

Podemos listar trabalhos de extração de texto usando um método de lista adequado.

In [30]:
extraction.list_jobs()

,EXTRACTION_ID,CREATED,STATUS
0,438124fb-61ef-49d7-9083-9fff1fb6f916,2025-05-13T20:26:11.606Z,running
1,58298596-5341-4623-a7d6-7b01943fff77,2025-05-13T20:08:10.201Z,completed
2,f274fdcf-c31b-401b-abea-2e4f6bc0c82f,2025-05-13T20:12:57.179Z,completed


Além disso, para obter detalhes de uma solicitação específica de extração de texto, execute o seguinte


In [42]:
extraction.get_job_details(extraction_id=extraction_job_id)

{'entity': {'document_reference': {'connection': {'id': '8a9a4e88-24df-4ddd-8ef1-7e1dc95b19b4'},
   'location': {'bucket': 'bucket-uxe54qdrdqc2c41',
    'file_name': 'RI-Magalu.png'},
   'type': 'connection_asset'},
  'parameters': {'create_embedded_images': 'disabled',
   'languages': ['pt'],
   'mode': 'standard',
   'output_dpi': 72,
   'output_tokens_and_bbox': True,
   'requested_outputs': ['plain_text']},
  'results': {'completed_at': '2025-05-13T20:29:11.081Z',
   'location': ['plain.txt'],
   'number_pages_processed': 1,
   'running_at': '2025-05-13T20:29:05.661Z',
   'status': 'completed'},
  'results_reference': {'connection': {'id': '8a9a4e88-24df-4ddd-8ef1-7e1dc95b19b4'},
   'location': {'bucket': 'bucket-uxe54qdrdqc2c41', 'file_name': 'plain.txt'},
   'type': 'connection_asset'},
  'steps': {'ocr': {'languages_list': ['pt']},
   'tables_processing': {'enabled': True}}},
 'metadata': {'created_at': '2025-05-13T20:29:02.927Z',
  'id': '69cc5101-8ea6-49c9-8fb8-8b8d984a2c02',


 Além disso, para excluir o trabalho de extração de texto, execute o método `delete_job()`.


## Análise dos resultados


Uma vez que a extração do trabalho seja concluída, podemos baixar o arquivo de resultados e processá-lo posteriormente.


In [ ]:
results_reference = extraction.get_results_reference(extraction_id=extraction_job_id)
results_reference.download(filename=results_file_name)

Extraí o resultado do OCR, removendo espaços vazios e convertendo para string

In [52]:
with open(results_file_name, "r", encoding="utf-8") as file:
    resultado_ocr = file.read()

print(resultado_ocr)

Divulgação de Resultados 4T24
      Var(%)      
Vendas Totais' (incluindo marketplace) 18.419,9 17.947,4 2,6% 65.330,9 63.056,3 3,6%
Receita Bruta 13.404,1 13.062,5 2,6% 47.277,0 45.591,0 3,7%
Receita Líquida 10.787,3 10.549,7 2,3% 38.038,1 36.768,1 3,5%
Lucro Bruto 3.244,0 3.192,8 1,6% 11.627,3 10.163,9 14,4%
Margem Bruta 30,1% 30,3% -0,2 pp 30,6% 27,6% 3,0 pp
EBITDA 842,4 548,6 53,6% 2.895,7 870,5 232,6%
Margem EBITDA 7,8% 5,2% 2,6 pp 7,6% 2,4% 5,2 pp
Lucro Líquido 294,8 212,2 38,9% 448,7 (979,1)  
Margem Líquida 2,7% 2,0% 0,7 pp 1,2% -2,7% 3,9 pp
Lucro Bruto - - Ajustado 3.244,0 3.192,8 1,6% 11.627,3 10.750,1 8,2%
Margem Bruta - Ajustado - 30,1% 30,3% -0,2 pp 30,6% 29,2% 1,4 pp
EBITDA - - Ajustado 846,2 756,5 11,9% 2.962,2 2.131,8 39,0%
Margem EBITDA Ajustado 7,8% 7,2% 0,6 pp 7,8% 5,8% 2,0 pp
Lucro Líquido - Ajustado 139,2 101,5 37,1% 276,7 (550,1)  
Margem Líquida - Ajustado - 1,3% 1,0% 0,3 pp 0,7% -1,5% 2,2 pp
Crescimento nas Vendas Mesmas Lojas Físicas 8,1% 3,7%   11,7% 4,7%  
C

### Perguntas e Respostas (Q&A) do artigo a partir do texto extraido por IOCR

In [53]:
input_question = "Qual foi o EBITDA em 4T23? "

prompt_text = f"""
Você é um assistente responsável por responder perguntas com base em um contexto fornecido. 
Este é seu contexto: {resultado_ocr}.
Pergunta: {input_question} 
Resposta:
"""


result = model_gen_ai.generate_text(prompt=prompt_text)

print(result)

R$ 548,6 milhões
